In [95]:
import numpy as np 
def createC1(dataSet):
   C1 = []
   for transaction in dataSet:
       for item in transaction:
           if not [item] in C1:
               C1.append([item]) #store all the item unrepeatly

   C1.sort()
   #return map(frozenset, C1)#frozen set, user can't change it.
   return list(map(frozenset, C1))

def scanD(D,Ck,minSupport):
#参数：数据集、候选项集列表 Ck以及感兴趣项集的最小支持度 minSupport
   ssCnt={}
   for tid in D:#遍历数据集
       for can in Ck:#遍历候选项
           if can.issubset(tid):#判断候选项中是否含数据集的各项
               #if not ssCnt.has_key(can): # python3 can not support
               if not can in ssCnt:
                   ssCnt[can]=1 #不含设为1
               else: ssCnt[can]+=1#有则计数加1
   numItems=float(len(D))#数据集大小
   retList = []#L1初始化
   supportData = {}#记录候选项中各个数据的支持度
   for key in ssCnt:
       support = ssCnt[key]/numItems#计算支持度
       if support >= minSupport:
           retList.insert(0,key)#满足条件加入L1中
       supportData[key] = support
   return retList, supportData

def aprioriGen(Lk, k): #组合，向上合并
   #creates Ck 参数：频繁项集列表 Lk 与项集元素个数 k
   retList = []
   lenLk = len(Lk)
   for i in range(lenLk):
       for j in range(i+1, lenLk): #两两组合遍历
           L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
           L1.sort(); L2.sort()
           if L1==L2: #若两个集合的前k-2个项相同时,则将两个集合合并
               retList.append(Lk[i] | Lk[j]) #set union
   return retList

In [96]:
def apriori(dataSet, minSupport = 0.5):
   C1 = createC1(dataSet)
   D = list(map(set, dataSet)) #python3
   L1, supportData = scanD(D, C1, minSupport)#单项最小支持度判断 0.5，生成L1
   L = [L1]
   k = 2
   while (len(L[k-2]) > 0):#创建包含更大项集的更大列表,直到下一个大的项集为空
       Ck = aprioriGen(L[k-2], k)#Ck
       Lk, supK = scanD(D, Ck, minSupport)#get Lk
       supportData.update(supK)
       L.append(Lk)
       k += 1
   L.pop()
   return L, supportData

def generateRules(L, supportData, items_num = None, minConf=0.7):
   #频繁项集列表、包含那些频繁项集支持数据的字典、最小可信度阈值
   bigRuleList = [] #存储所有的关联规则
   for i in range(1, len(L)):  #只获取有两个或者更多集合的项目，从1,即第二个元素开始，L[0]是单个元素的
       # 两个及以上的才可能有关联一说，单个元素的项集不存在关联问题
       for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            #该函数遍历L中的每一个频繁项集并对每个频繁项集创建只包含单个元素集合的列表H1
            if items_num is None:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)# 调用函数2
            elif items_num < 2:
                print('items_num must be at least 2!')
                break
            elif (i+1 == items_num):
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)# 调用函数2
   return bigRuleList

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
   #针对项集中只有两个元素时，计算可信度
   prunedH = []#返回一个满足最小可信度要求的规则列表
   for conseq in H:#后件，遍历 H中的所有项集并计算它们的可信度值
       conf = supportData[freqSet]/supportData[freqSet-conseq] #可信度计算，结合支持度数据
       if conf >= minConf:
           brl.append((freqSet-conseq, conseq, conf))#添加到规则里，brl 是前面通过检查的 bigRuleList
           prunedH.append(conseq)#同样需要放入列表到后面检查
   return prunedH


def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
   #参数:一个是频繁项集,另一个是可以出现在规则右部的元素列表 H
   m = len(H[0])
   if (len(freqSet)) > (m + 1): #频繁项集元素数目大于单个集合的元素数
       Hmp1 = aprioriGen(H, m+1)#存在不同顺序、元素相同的集合，合并具有相同部分的集合
       Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)#计算可信度
       if (len(Hmp1) > 1):    
       #满足最小可信度要求的规则列表多于1,则递归来判断是否可以进一步组合这些规则
           rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf) 

In [97]:
import pandas as pd

supermarket_data = pd.read_csv('supermarket.csv') 
supermarket_data.dropna(inplace = True) 
supermarket_data.head()

,'grocery misc','baby needs','bread and cake','baking needs',coupons,juice-sat-cord-ms,tea,biscuits,'canned fish-meat','canned fruit',...,'750ml red nz','750ml white imp','750ml red imp','sparkling nz','sparkling imp','brew kits/accesry','port and sherry',ctrled label wine','non host support',total
0,?,t,t,t,?,t,?,t,?,?,...,?,?,?,?,?,?,?,?,?,high
1,?,?,?,?,?,?,?,?,t,t,...,?,?,?,?,?,?,?,?,?,low
2,?,?,t,t,?,t,?,t,?,t,...,?,?,?,?,?,?,?,?,?,low
3,?,?,t,t,?,t,?,t,?,?,...,?,?,?,?,?,?,?,?,?,low
4,?,?,t,t,?,t,t,?,?,?,...,?,?,?,?,?,?,?,?,?,low


In [98]:
supermarket_data.drop(columns = 'total',inplace=True)
columns_list = list(supermarket_data)
for i in range(len(supermarket_data)):
    for j in range(len(supermarket_data.columns)):
        if supermarket_data.iloc[i,j] == 't':
            supermarket_data.iloc[i,j] = columns_list[j]

supermarket_data = supermarket_data.values.tolist()
for i in range(len(supermarket_data)):
    supermarket_data[i] = list(filter(lambda x : x != '?', supermarket_data[i]))

In [99]:
item_list = supermarket_data
frequent_itemsets, supportdata = apriori(item_list,minSupport = 0.15)

In [100]:
frequent_itemsets

[[frozenset({"'cigs-tobacco pkts'"}),
  frozenset({"'soft drinks'"}),
  frozenset({'tea'}),
  frozenset({'poultry'}),
  frozenset({'potatoes'}),
  frozenset({"'small goods2'"}),
  frozenset({"'dental needs'"}),
  frozenset({'jams-spreads'}),
  frozenset({"'prepared meals'"}),
  frozenset({"'breakfast food'"}),
  frozenset({'stationary'}),
  frozenset({'puddings-deserts'}),
  frozenset({'beef'}),
  frozenset({"'dairy foods'"}),
  frozenset({'haircare'}),
  frozenset({'deodorants-soap'}),
  frozenset({"'pet foods'"}),
  frozenset({"'laundry needs'"}),
  frozenset({"'canned fruit'"}),
  frozenset({"'canned fish-meat'"}),
  frozenset({'wrapping'}),
  frozenset({'vegetables'}),
  frozenset({'sauces-gravy-pkle'}),
  frozenset({'milk-cream'}),
  frozenset({'margarine'}),
  frozenset({'juice-sat-cord-ms'}),
  frozenset({'fruit'}),
  frozenset({'confectionary'}),
  frozenset({'coffee'}),
  frozenset({'cleaners-polishers'}),
  frozenset({'cheese'}),
  frozenset({'biscuits'}),
  frozenset({"'tiss

In [101]:
supportdata_order=sorted(supportdata.items(),key=lambda x:x[1],reverse=True)
supportdata_order

[(frozenset({"'bread and cake'"}), 0.7196281884997838),
 (frozenset({'fruit'}), 0.6400778210116731),
 (frozenset({'vegetables'}), 0.6398616515348032),
 (frozenset({'milk-cream'}), 0.6353220925205361),
 (frozenset({"'baking needs'"}), 0.6039775183744055),
 (frozenset({"'frozen foods'"}), 0.587116299178556),
 (frozenset({'biscuits'}), 0.562905317769131),
 (frozenset({'juice-sat-cord-ms'}), 0.53220925205361),
 (frozenset({"'bread and cake'", 'milk-cream'}), 0.5051880674448768),
 (frozenset({"'party snack foods'"}), 0.5034587116299178),
 (frozenset({"'bread and cake'", 'fruit'}), 0.5023778642455685),
 (frozenset({"'bread and cake'", 'vegetables'}), 0.49654128837008216),
 (frozenset({'margarine'}), 0.4943795936013835),
 (frozenset({"'tissues-paper prd'"}), 0.485516645049719),
 (frozenset({'fruit', 'vegetables'}), 0.47686986597492437),
 (frozenset({'sauces-gravy-pkle'}), 0.475789018590575),
 (frozenset({"'baking needs'", "'bread and cake'"}), 0.47341115434500647),
 (frozenset({"'bread and ca

In [102]:
supportdata_list = [[]for i in range(len(supportdata_order))]
for i in range(len(supportdata_order)):
    for j in supportdata_order[i][0]:
        supportdata_list[i].append(j)
# print(supportdata_list)

In [103]:
supportdata_dict = {}
for i in range(len(supportdata_order)):
    supportdata_dict[','.join(str(j) for j in supportdata_list[i])] = supportdata_order[i][1]
# print(supportdata_dict)


In [104]:
supportdata_df = pd.DataFrame([supportdata_dict])
supportdata_df = supportdata_df.T
supportdata_df.columns = ['support']
supportdata_df.to_csv('supportdata.csv')
supportdata_df

,support
'bread and cake',0.719628
fruit,0.640078
vegetables,0.639862
milk-cream,0.635322
'baking needs',0.603978
...,...
mutton,0.004972
'sparkling imp',0.004972
chickens,0.004540
salads,0.001297


In [105]:
#满足最小可信度要求的频繁项目集
ConfList = generateRules(frequent_itemsets, supportdata, items_num = 3, minConf=0.8)
ConfList

[(frozenset({"'prepared meals'", 'milk-cream'}),
  frozenset({"'bread and cake'"}),
  0.8389021479713603),
 (frozenset({"'prepared meals'", 'vegetables'}),
  frozenset({"'bread and cake'"}),
  0.8302752293577982),
 (frozenset({"'prepared meals'", 'fruit'}),
  frozenset({"'bread and cake'"}),
  0.844626168224299),
 (frozenset({"'laundry needs'", "'tissues-paper prd'"}),
  frozenset({"'bread and cake'"}),
  0.8216374269005848),
 (frozenset({"'frozen foods'", "'soft drinks'"}),
  frozenset({"'bread and cake'"}),
  0.8023715415019762),
 (frozenset({'milk-cream', 'stationary'}),
  frozenset({"'bread and cake'"}),
  0.813953488372093),
 (frozenset({"'dairy foods'", 'milk-cream'}),
  frozenset({"'bread and cake'"}),
  0.8255915863277826),
 (frozenset({"'laundry needs'", "'party snack foods'"}),
  frozenset({"'bread and cake'"}),
  0.8279220779220778),
 (frozenset({"'canned fruit'", "'frozen foods'"}),
  frozenset({"'bread and cake'"}),
  0.8297180043383947),
 (frozenset({"'laundry needs'", 'f

In [106]:
ConfList_list = [[]for i in range(len(ConfList))]
for i in range(len(ConfList)):
    m = 0
    while m < len(ConfList[i])-1:
        for j in ConfList[i][m]:
            ConfList_list[i].append(j)
        m += 1     
ConfList_list

[["'prepared meals'", 'milk-cream', "'bread and cake'"],
 ['vegetables', "'prepared meals'", "'bread and cake'"],
 ["'prepared meals'", 'fruit', "'bread and cake'"],
 ["'tissues-paper prd'", "'laundry needs'", "'bread and cake'"],
 ["'soft drinks'", "'frozen foods'", "'bread and cake'"],
 ['stationary', 'milk-cream', "'bread and cake'"],
 ["'dairy foods'", 'milk-cream', "'bread and cake'"],
 ["'laundry needs'", "'party snack foods'", "'bread and cake'"],
 ["'canned fruit'", "'frozen foods'", "'bread and cake'"],
 ["'laundry needs'", 'fruit', "'bread and cake'"],
 ["'laundry needs'", 'biscuits', "'bread and cake'"],
 ["'breakfast food'", 'vegetables', "'bread and cake'"],
 ["'breakfast food'", 'vegetables', 'fruit'],
 ['jams-spreads', 'fruit', "'bread and cake'"],
 ['sauces-gravy-pkle', "'pet foods'", "'bread and cake'"],
 ["'pet foods'", 'vegetables', "'bread and cake'"],
 ["'pet foods'", 'milk-cream', "'bread and cake'"],
 ["'pet foods'", 'fruit', "'bread and cake'"],
 ["'canned fruit

In [107]:
ConfList_df = pd.DataFrame(ConfList_list)
for i in range(1,len(ConfList_df.columns)): 
    for j in range(len(ConfList_df)): 
        if ConfList_df.iloc[j,i] is not None:
            ConfList_df.iloc[j,0] = ConfList_df.iloc[j,0]+', ' + ConfList_df.iloc[j,i]
ConfList_df = ConfList_df[[0]]
ConfList_df

ConfList_df.columns = ['iterms']
ConfList_df['conf'] = 0
for i in range(len(ConfList)):
    ConfList_df.iloc[i,1] = ConfList[i][-1]
ConfList_df.sort_values(by='conf',ascending=False,inplace = True)
ConfList_df = ConfList_df.reset_index(drop=True)
ConfList_df

,iterms,conf
0,"'canned fruit', milk-cream, 'bread and cake'",0.865688
1,"jams-spreads, milk-cream, 'bread and cake'",0.858783
2,"'canned fruit', biscuits, 'bread and cake'",0.854875
3,"'canned fruit', fruit, 'bread and cake'",0.853632
4,"vegetables, 'small goods', 'bread and cake'",0.851582
...,...,...
180,"'soft drinks', 'frozen foods', 'bread and cake'",0.802372
181,"'baking needs', confectionary, 'bread and cake'",0.802007
182,"'breakfast food', vegetables, fruit",0.801724
183,"confectionary, 'frozen foods', 'bread and cake'",0.801477
